In [ ]:
# Reference: https://pypi.org/project/diffusers/

#### Setup

In [ ]:
!pip install diffusers[torch] transformers accelerate

In [ ]:
!apt-get install git-lfs

In [ ]:
%mkdir -p /pt/diffusers/stable-diffusion-v1-5
%cd /pt/diffusers/stable-diffusion-v1-5

!git lfs install

%cd /pt/diffusers/
#!git lfs

In [ ]:
!git clone https://huggingface.co/runwayml/stable-diffusion-v1-5

In [ ]:
import torch

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"

#### Text-to-Image generation with Stable Diffusion

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

In [ ]:
from IPython import display
display.display(image)

In [ ]:
# Optionally save output
image.save("astronaut_rides_horse.png")

 #### Image-to-Image text-guided generation with Stable Diffusion

In [ ]:
import os.path
url, filename = ("https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg", "sketch-mountains-input.jpg")

if os.path.isfile(filename):
    print("Image file exists")
else:
    import urllib.request
    urllib.request.urlretrieve(url, filename)
    print("Image file downloaded")
    

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline

from PIL import Image
init_image = Image.open(filename).convert("RGB")
init_image = init_image.resize((512, 512))
#display.display(init_image)

prompt = "A fantasy landscape, trending on artstation"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to(device)

In [ ]:
generator = torch.Generator(device=device).manual_seed(1024)
images = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5, generator=generator, num_inference_steps=500).images

from IPython import display
display.display(images[0])